In [ ]:
import sys, os
sys.path.append("..")   # eine Ebene hoch

from model.sim_model import load_scenario, simulate_load_profile

scenario = load_scenario("../scenarios/office.yaml")

timestamps, load_kw = simulate_load_profile(scenario)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({"timestamp": timestamps, "load_kw": load_kw})
df = df.set_index("timestamp")

df["load_kw"].plot(figsize=(14,4))
plt.grid(True)
plt.show()
